In [ ]:
#| default_exp datasources
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [ ]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# Fontes de Dados
> Módulo para encapsular a extração e processamento das diferentes fontes de dados

In [ ]:
#| export
import os
import re
from dataclasses import dataclass
from decimal import Decimal, getcontext
from functools import cached_property
from typing import Tuple, Union

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.foundation import GetAttr
from fastcore.xtras import Path
from pyarrow import ArrowInvalid, ArrowTypeError

from extracao.connectors import MongoDB, SQLServer
from extracao.constants import (
    BW,
    BW_MAP,
    COLS_SMP,
    COLS_SRD,
    COLS_TELECOM,
    COLUNAS,
    MONGO_SMP,
    MONGO_SRD,
    MONGO_TELECOM,
    PROJECTION_SRD,
    RE_BW,
)

In [ ]:
#| export
getcontext().prec = 5
load_dotenv(find_dotenv())

True

In [ ]:
#| hide: true
#| eval:false
__file__ = Path.cwd().parent / 'extracao' / 'datasources.py'

In [ ]:
#| export

SQLSERVER_PARAMS = dict(
    driver="{ODBC Driver 17 for SQL Server}",
    server="ANATELBDRO05",
    database="SITARWEB",
    trusted_conn=True,
    mult_results=True,
    username=None,
    password=None,
    timeout=1000,
)

MONGO_URI = os.environ.get("MONGO_URI")


@dataclass
class Base:
    folder: Union[str, Path] = Path(__file__).parent / "dados"

    def _read(self, stem: str) -> pd.DataFrame:
        """Lê o dataframe formado por self.folder / self.stem.parquet.gzip"""
        file = Path(f"{self.folder}/{stem}.parquet.gzip")
        try:
            df = pd.read_parquet(file)
        except (ArrowInvalid, FileNotFoundError) as e:
            raise e(f"Error when reading {file}") from e
        return df
    
    @cached_property
    def df(self):
        raise NotImplementedError
    
    @cached_property    
    def extract(self):
        raise NotImplementedError

    def _format(self, df: pd.DataFrame) -> pd.DataFrame:
        raise NotImplementedError

    def update(self):
        raise NotImplementedError

    def save(self, folder: Union[str, Path]) -> pd.DataFrame:
        """Format, Save and return a dataframe"""
        df = self.df.astype("string")
        df = df.drop_duplicates(keep="first", ignore_index=True)
        try:
            file = Path(f"{folder}/{self.stem}.parquet.gzip")
            df.to_parquet(file, compression="gzip", index=False)
        except (ArrowInvalid, ArrowTypeError) as e:
            raise e(f"Não foi possível salvar o arquivo parquet {file}") from e
        return df


In [ ]:
#| export
class Sitarweb(Base, GetAttr):
    def __init__(self, sql_params: dict = SQLSERVER_PARAMS):
        self.default = SQLServer(sql_params)

In [ ]:
#| export
class Mosaico(Base, GetAttr):
    def __init__(self, mongo_uri: str = MONGO_URI):
        self.database = "sms"
        self.default = MongoDB(mongo_uri)

    def _extract(self, collection: str, pipeline: list):
        client = self.connect()
        database = client[self.database]
        collection = database[collection]
        result = collection.aggregate(pipeline)
        df = pd.DataFrame(list(result))
        df = df.drop(columns=["_id"])
        return df.astype("string")

    @staticmethod
    def parse_bw(
        bw: str,  # Designação de Emissão (Largura + Classe) codificada como string
    ) -> Tuple[str, str]:  # Largura e Classe de Emissão
        """Parse the bandwidth string"""
        if match := re.match(RE_BW, bw):
            multiplier = BW[match[2]]
            if mantissa := match[3]:
                number = float(f"{match[1]}.{mantissa}")
            else:
                number = float(match[1])
            classe = match[4]
            return str(multiplier * number), str(classe)
        return pd.NA, pd.NA

    @staticmethod
    def split_designacao(
        df: pd.DataFrame,  # DataFrame com coluna original DesignacaoEmissao
    ) -> (
        pd.DataFrame
    ):  # DataFrame com novas colunas Largura_Emissão(kHz) e Classe_Emissão
        """Parse a bandwidth string to extract the numerical component and a character class"""
        df["Designação_Emissão"] = (
            df["Designação_Emissão"].str.replace(",", " ").str.strip().str.upper()
        )
        df["Designação_Emissão"] = df["Designação_Emissão"].str.split(" ")
        df = df.explode("Designação_Emissão")
        df = df[df["Designação_Emissão"] != "/"]  # Removes empty rows
        df[["Largura_Emissão(kHz)", "Classe_Emissão"]] = (
            df["Designação_Emissão"].apply(Mosaico.parse_bw).tolist()
        )
        df[["Largura_Emissão(kHz)", "Classe_Emissão"]] = df[
            ["Largura_Emissão(kHz)", "Classe_Emissão"]
        ].astype("string")
        return df.drop("Designação_Emissão", axis=1)
    
    def extract(self)->pd.DataFrame:
        raise NotImplementedError


In [ ]:
#| export
class SRD(GetAttr):
    """Classe para encapsular a lógica de extração de Radiodifusão"""

    def __init__(self, mongo_uri: str = MONGO_URI) -> None:
        self.stem = 'srd'
        self.default = Mosaico(mongo_uri)
        self.collection = "srd"
        self.query = MONGO_SRD
        self.projection = PROJECTION_SRD
        self.columns = COLS_SRD

    @cached_property
    def df(self)->pd.DataFrame:
        return self._read(self.stem)
    
    @cached_property
    def extract(self)->pd.DataFrame:
        pipeline = [
            # match the documents that satisfy your query
            {"$match": self.query},
            # project the fields that you want to keep
            {"$project": self.projection}
        ]
        df =  self._extract(self.collection, pipeline)
        df.loc[df['estacao'] == '[]', 'estacao'] = '{}'
        cols = ['srd_planobasico', 'estacao', 'habilitacao', 'Status']
        for col in cols:            
            df = df.join(pd.json_normalize(df[col].apply(eval)))
        df = df.drop(columns=cols)
        #Substitui strings vazias e somente com espaços por nulo
        return df.replace(r'^\s*$', pd.NA, regex=True)


    def _format(
        self,
        df: pd.DataFrame,  # DataFrame com os dados de Estações e Plano_Básico mesclados
    ) -> pd.DataFrame:  # DataFrame com os dados mesclados e limpos
        """Clean the merged dataframe with the data from the MOSAICO page"""
        df = df.rename(columns=self.columns)
        df = df[
            df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)
        ].reset_index(drop=True)
        df["Frequência"] = df.Frequência.astype("string").str.replace(",", ".").astype('float')
        df = df.dropna(subset='Frequência', ignore_index=True)
        df.loc[df['Num_Serviço'] == "205", "Frequência"] = df.loc[df['Num_Serviço'] == "205", "Frequência"
        ].apply(lambda x: Decimal(x) / Decimal(1000))
        df["Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
        df["Fonte"] = "MOS"
        df["Num_Serviço"] = df["Num_Serviço"].fillna("")
        df["Designação_Emissão"] = (
            df.Num_Serviço.astype("string").fillna("").map(BW_MAP)
        )
        df = self.split_designacao(df)
        df["Multiplicidade"] = 1
        return df.loc[:, COLUNAS]

    def update(self):
        self.df = self._format(self.extract)


In [ ]:
#| eval: false
srd = SRD()

In [ ]:
#| eval: false
srd.df.tail()

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
31030,491.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446491632,<NA>,Arraial do Cabo,3300258,RJ,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
31031,485.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446491802,<NA>,Santana,1600600,AP,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
31032,659.0,TV CIDADE PRODUCOES LTDA,50446492108,<NA>,São Mateus do Maranhão,2111508,MA,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
31033,659.0,TV CIDADE PRODUCOES LTDA,50446492523,<NA>,Peritoró,2108454,MA,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
31034,587.0,TV CIDADE PRODUCOES LTDA,50446492876,<NA>,Santa Inês,2109908,MA,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1


In [ ]:
#| eval: false
srd.extract.tail()

,NumServico,stnClass,frequency,licensee,NumFistel,NomeMunicipio,CodMunicipio,SiglaUF,NumEstacao,MedLatitudeDecimal,MedLongitudeDecimal,DataValFreq,state
35585,801,C,491.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446491632,Arraial do Cabo,3300258,RJ,NaN,NaN,NaN,NaN,TV-C1
35586,801,C,485.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446491802,Santana,1600600,AP,NaN,NaN,NaN,NaN,TV-C1
35587,801,C,659.0,TV CIDADE PRODUCOES LTDA,50446492108,São Mateus do Maranhão,2111508,MA,NaN,NaN,NaN,NaN,TV-C1
35588,801,C,659.0,TV CIDADE PRODUCOES LTDA,50446492523,Peritoró,2108454,MA,NaN,NaN,NaN,NaN,TV-C1
35589,801,C,587.0,TV CIDADE PRODUCOES LTDA,50446492876,Santa Inês,2109908,MA,NaN,NaN,NaN,NaN,TV-C1


In [ ]:
#| eval: false
srd.update()
srd.df

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,207.0,REDE DE COMUNICACOES ACREANA LTDA,50442889933,NaN,Cruzeiro do Sul,1200203,AC,NaN,NaN,A,248,,6000.0,<NA>,TV-C1,MOS,1
1,539.0,X-MEDIAGROUP S.A.,50410887137,<NA>,Mâncio Lima,1200336,AC,NaN,NaN,C,248,,6000.0,<NA>,TV-C1,MOS,1
2,79.0,TELEVISAO OESTE BAIANO LTDA,06030116240,322647029,Barreiras,2903201,BA,-12.1013888888888333,-44.9936111111110000,A,248,,6000.0,2023-12-31,TV-C4,MOS,1
3,69.0,TELEVISAO SANTA CRUZ LTDA,06020355110,322623553,Itabuna,2914802,BA,-14.7794444444443333,-39.2622222222221666,A,248,,6000.0,2023-12-31,TV-C4,MOS,1
4,177.0,TV CABRALIA LTDA,06020354903,322623537,Itabuna,2914802,BA,-14.7833333333333333,-39.2833333333333333,B,248,,6000.0,2023-12-31,TV-C4,MOS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31056,491.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446491632,NaN,Arraial do Cabo,3300258,RJ,NaN,NaN,C,801,,5700.0,<NA>,TV-C1,MOS,1
31057,485.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446491802,NaN,Santana,1600600,AP,NaN,NaN,C,801,,5700.0,<NA>,TV-C1,MOS,1
31058,659.0,TV CIDADE PRODUCOES LTDA,50446492108,NaN,São Mateus do Maranhão,2111508,MA,NaN,NaN,C,801,,5700.0,<NA>,TV-C1,MOS,1
31059,659.0,TV CIDADE PRODUCOES LTDA,50446492523,NaN,Peritoró,2108454,MA,NaN,NaN,C,801,,5700.0,<NA>,TV-C1,MOS,1


In [ ]:
#| export
class Telecom(GetAttr):
    """Classe para encapsular a lógica de extração dos serviços de Telecomunições distintos de SMP"""

    def __init__(self, mongo_uri: str = MONGO_URI) -> None:
        self.default = Mosaico(mongo_uri)
        self.collection = "licenciamento"
        self.query = MONGO_TELECOM
        self.columns = COLS_TELECOM

    def format(self):
        pass


In [ ]:
#| export

class SMP(GetAttr):
    """Classe para encapsular a lógica de extração do SMP"""

    def __init__(self, mongo_uri: str = MONGO_URI) -> None:
        self.default = Mosaico(mongo_uri)
        self.collection = "licenciamento"
        self.query = MONGO_SMP
        self.columns = COLS_SMP